In [1]:
import os
import sys

import numpy as np
import pandas as pd


In [2]:
# lib_path = os.path.abspath(os.path.pardir) # same as os.path.abspath("..")
lib_path = "/projects/timshel/sc-genetics/sc-genetics/src/lib"
sys.path.insert(1, lib_path)
from sem_pre_calculation import *

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Constants

In [4]:
out_prefix = "tabula_muris"

### Read data

In [5]:
### Metadata
file_metadata = "/scratch/data-for_fast_access/pub-others/tabula_muris_180920/tabula_muris.metadata.csv"
df_metadata = pd.read_csv(file_metadata, index_col=False)
df_metadata.head()

,cell_id,nGene,nReads,tissue,subtissue_clean,celltype,tissue_celltype,tissue_subtissue_celltype
0,A1.B000126.3_39_F.1.1,3125,599257,Skin,Telogen,epidermal cell,Skin.epidermal cell,Skin.Telogen.epidermal cell
1,A1.B003283.3_38_F.1.1,5543,2585048,Skin,Telogen,epidermal cell,Skin.epidermal cell,Skin.Telogen.epidermal cell
2,A1.MAA000435.3_10_M.1.1,5023,1748535,Skin,Anagen,basal cell of epidermis,Skin.basal cell of epidermis,Skin.Anagen.basal cell of epidermis
3,A1.MAA000549.3_8_M.1.1,3846,309793,Skin,Anagen,epidermal cell,Skin.epidermal cell,Skin.Anagen.epidermal cell
4,A1.MAA000614.3_10_M.1.1,3249,1044110,Skin,Telogen,basal cell of epidermis,Skin.basal cell of epidermis,Skin.Telogen.basal cell of epidermis


In [7]:
### Data
file_data = "/scratch/data-for_fast_access/pub-others/tabula_muris_180920/tabula_muris.umi.csv.gz"
df_data = pd.read_csv(file_data, index_col=False) # this takes 12-14 min for tabula_muris! (Pandas is slow!)

In [8]:
df_data.set_index("gene", inplace=True) # set index
df_data.head()

,A1.B000126.3_39_F.1.1,A1.B003283.3_38_F.1.1,A1.MAA000435.3_10_M.1.1,A1.MAA000549.3_8_M.1.1,A1.MAA000614.3_10_M.1.1,A1.MAA000938.3_8_M.1.1,A10.B003283.3_38_F.1.1,A11.B000126.3_39_F.1.1,A12.B000126.3_39_F.1.1,A12.B003283.3_38_F.1.1,...,O6.MAA001847.3_39_F.1.1,O7.MAA001847.3_39_F.1.1,P2.MAA001847.3_39_F.1.1,P3.MAA000839.3_11_M.1.1,P4.MAA000526.3_9_M.1.1,P4.MAA000839.3_11_M.1.1,P5.MAA000526.3_9_M.1.1,P5.MAA001847.3_39_F.1.1,P6.MAA001847.3_39_F.1.1,P9.MAA001847.3_39_F.1.1
gene,,,,,,,,,,,,,,,,,,,,,
0610005C13Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610007C21Rik,265,1026,0,48,26,87,36,245,0,245,...,74,0,164,0,51,315,1,0,193,0
0610007L01Rik,1,35,4,0,186,2,0,0,0,0,...,72,4,7,0,0,0,0,15,0,33
0610007N19Rik,0,0,0,0,0,0,0,0,0,0,...,11,0,0,0,0,3,0,1,0,0
0610007P08Rik,0,0,0,10,0,0,110,0,0,0,...,0,0,0,0,0,0,0,0,53,0


In [11]:
### [*IMPORTANT*] Check that all metadata cell_ids are identical to data columnnames. 
### We need to ensure this before we can use the metadata cell-types as annotations.
np.all(df_data.columns.values == df_metadata["cell_id"].values) # ---> True

True

### CTC log normalize

In [12]:
df_ctc_log = ctc_log_normalize(df_data)

Performning common transcript count (ctc) normalization and log-transformation on input data


### Set annotations

In [17]:
annotations = df_metadata["tissue_celltype"].values
annotations[:5]

array(['Skin.epidermal cell', 'Skin.epidermal cell',
       'Skin.basal cell of epidermis', 'Skin.epidermal cell',
       'Skin.basal cell of epidermis'], dtype=object)

### Anova

In [19]:
df_anova = calculate_anova_sporadically_expressed_genes(df_ctc_log, annotations, out_prefix)
# Number of genes sporadically expressed (pvalue > 0.00001, Skene cut-off): 2365

Splitting data frame into annotation groups
Splitting annotation #1/#115 into group
Splitting annotation #2/#115 into group
Splitting annotation #3/#115 into group
Splitting annotation #4/#115 into group
Splitting annotation #5/#115 into group
Splitting annotation #6/#115 into group
Splitting annotation #7/#115 into group
Splitting annotation #8/#115 into group
Splitting annotation #9/#115 into group
Splitting annotation #10/#115 into group
Splitting annotation #11/#115 into group
Splitting annotation #12/#115 into group
Splitting annotation #13/#115 into group
Splitting annotation #14/#115 into group
Splitting annotation #15/#115 into group
Splitting annotation #16/#115 into group
Splitting annotation #17/#115 into group
Splitting annotation #18/#115 into group
Splitting annotation #19/#115 into group
Splitting annotation #20/#115 into group
Splitting annotation #21/#115 into group
Splitting annotation #22/#115 into group
Splitting annotation #23/#115 into group
Splitting annotation #

/tools/anaconda/3-4.4.0/envs/py3_anaconda3_PT180510/lib/python3.6/site-packages/scipy/stats/stats.py:2933: RuntimeWarning: invalid value encountered in double_scalars
  f = msb / msw


gene 500 out of 23341
gene 600 out of 23341
gene 700 out of 23341
gene 800 out of 23341
gene 900 out of 23341
gene 1000 out of 23341
gene 1100 out of 23341
gene 1200 out of 23341
gene 1300 out of 23341
gene 1400 out of 23341
gene 1500 out of 23341
gene 1600 out of 23341
gene 1700 out of 23341
gene 1800 out of 23341
gene 1900 out of 23341
gene 2000 out of 23341
gene 2100 out of 23341
gene 2200 out of 23341
gene 2300 out of 23341
gene 2400 out of 23341
gene 2500 out of 23341
gene 2600 out of 23341
gene 2700 out of 23341
gene 2800 out of 23341
gene 2900 out of 23341
gene 3000 out of 23341
gene 3100 out of 23341
gene 3200 out of 23341
gene 3300 out of 23341
gene 3400 out of 23341
gene 3500 out of 23341
gene 3600 out of 23341
gene 3700 out of 23341
gene 3800 out of 23341
gene 3900 out of 23341
gene 4000 out of 23341
gene 4100 out of 23341
gene 4200 out of 23341
gene 4300 out of 23341
gene 4400 out of 23341
gene 4500 out of 23341
gene 4600 out of 23341
gene 4700 out of 23341
gene 4800 out of

In [22]:
df_anova.head()
### Define sporatically expressed genes
# genes_sporadically = df_anova[df_anova['pvalue'] > 0.00001].index
# genes_sporadically

,pvalue,statistic
gene,,
4930524C18Rik,1.0,0.145111
Tmem225,1.0,0.181884
Mir1948,1.0,0.246055
7420426K07Rik,1.0,0.243265
Mir1912,1.0,0.222707


In [23]:
df_anova.to_csv("{}.pre_calc.sporadically_expressed_genes.anova.csv.gz".format(out_prefix), compression="gzip")

### SEM pre-calculation

In [28]:
(df_frac, df_mu, df_var, df_n) = calculate_per_anno_summary_stats(df_ctc_log, annotations, out_prefix, permute_annotations=False)

Running: #1/#115 | Bladder.bladder cell
Running: #2/#115 | Bladder.bladder urothelial cell
Running: #3/#115 | Brain_Myeloid.macrophage
Running: #4/#115 | Brain_Myeloid.microglial cell
Running: #5/#115 | Brain_Non-Myeloid.Bergmann glial cell
Running: #6/#115 | Brain_Non-Myeloid.astrocyte
Running: #7/#115 | Brain_Non-Myeloid.brain pericyte
Running: #8/#115 | Brain_Non-Myeloid.endothelial cell
Running: #9/#115 | Brain_Non-Myeloid.neuron
Running: #10/#115 | Brain_Non-Myeloid.oligodendrocyte
Running: #11/#115 | Brain_Non-Myeloid.oligodendrocyte precursor cell
Running: #12/#115 | Fat.B cell
Running: #13/#115 | Fat.T cell
Running: #14/#115 | Fat.endothelial cell
Running: #15/#115 | Fat.mesenchymal stem cell of adipose
Running: #16/#115 | Fat.myeloid cell
Running: #17/#115 | Fat.natural killer cell
Running: #18/#115 | Fat.unknown cell type
Running: #19/#115 | Heart.cardiac muscle cell
Running: #20/#115 | Heart.endocardial cell
Running: #21/#115 | Heart.endothelial cell
Running: #22/#115 | Hear

In [29]:
(df_frac_null, df_mu_null, df_var_null, df_n_null) = calculate_per_anno_summary_stats(df_ctc_log, annotations, out_prefix, permute_annotations=True)

Doing null computation. Permuting labels with seed(1).
Running: #1/#115 | Bladder.bladder cell
Running: #2/#115 | Bladder.bladder urothelial cell
Running: #3/#115 | Brain_Myeloid.macrophage
Running: #4/#115 | Brain_Myeloid.microglial cell
Running: #5/#115 | Brain_Non-Myeloid.Bergmann glial cell
Running: #6/#115 | Brain_Non-Myeloid.astrocyte
Running: #7/#115 | Brain_Non-Myeloid.brain pericyte
Running: #8/#115 | Brain_Non-Myeloid.endothelial cell
Running: #9/#115 | Brain_Non-Myeloid.neuron
Running: #10/#115 | Brain_Non-Myeloid.oligodendrocyte
Running: #11/#115 | Brain_Non-Myeloid.oligodendrocyte precursor cell
Running: #12/#115 | Fat.B cell
Running: #13/#115 | Fat.T cell
Running: #14/#115 | Fat.endothelial cell
Running: #15/#115 | Fat.mesenchymal stem cell of adipose
Running: #16/#115 | Fat.myeloid cell
Running: #17/#115 | Fat.natural killer cell
Running: #18/#115 | Fat.unknown cell type
Running: #19/#115 | Heart.cardiac muscle cell
Running: #20/#115 | Heart.endocardial cell
Running: #21